In [ ]:
"""baselines (1).ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/126HO5oSAEjWQF_--qKdMRfa-K9QK4OmH
"""

In [ ]:
import itertools
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import torch
import torch.nn.functional as F

In [ ]:
from scipy.stats import beta
import torch.nn as nn

In [ ]:
import scipy.integrate as integrate
import scipy.special as special

In [ ]:
class ExpoAlloc:
  
    def __init__(self, lbd, alpha, thetamax):
      self.lbd = lbd
      self.alpha = alpha
      self.thetamax = thetamax
      self.vstar = -np.log(1-self.thetamax)/self.lbd
    
    def phi(self,x):
      return x-1

    def alloc(self,arr):
      v2r = self.phi(arr[:,1])
      v1r = self.phi(arr[:,0])/self.alpha
      cpr = np.maximum(v1r,self.phi(self.vstar))
      return (v2r<=cpr).astype(int)

    def F(self,x):
      return np.power(np.e,-self.lbd*x)
    
    def interimalloc(self,x):
       return -self.alpha + self.alpha * (1-np.power(np.e, -self.lbd*(1+self.alpha*(x-1))))*int(x>self.vstar)+max(1-np.power(np.e,-self.lbd*self.vstar), 1-np.power(np.e,-self.lbd*(1+(x-1)/self.alpha)))
    
    def scipyprice(self,x):
      return x*self.interimalloc(x)-integrate.quad(lambda x: self.interimalloc(x), 0, x)[0]
    
    def error(self,x):
      return integrate.quad(lambda x: self.interimalloc(x), 0, x)[1]

    def sample(self,batch_size):
      return np.random.exponential(scale = self.lbd, size = (batch_size))

    def price(self,x):
      breakpoint1 = self.vstar
      breakpoint2 = 1+self.alpha*(self.vstar-1)
      if breakpoint1 <= breakpoint2:
        if x <= breakpoint1:
          return 0
        elif x > breakpoint1 and x <= breakpoint2:
          fstpart = -((1-self.alpha-np.power(np.e,-self.lbd*self.vstar))*breakpoint1)
          return self.interimalloc(x)*x+fstpart-np.power(np.e,self.lbd*(-1+self.alpha-self.alpha*x))/self.lbd + np.power(np.e,self.lbd*(-1+self.alpha-self.alpha*breakpoint1))/self.lbd-x+breakpoint1-np.power(np.e,-self.lbd*self.vstar)*(breakpoint1-x)
        else:
          fstpart = -((1-self.alpha-np.power(np.e,-self.lbd*self.vstar))*breakpoint1)
          sndpart = -np.power(np.e,self.lbd*(-1+self.alpha-self.alpha*breakpoint2))/self.lbd + np.power(np.e,self.lbd*(-1+self.alpha-self.alpha*breakpoint1))/self.lbd-breakpoint2+breakpoint1-np.power(np.e,-self.lbd*self.vstar)*(breakpoint1-breakpoint2)
          return self.interimalloc(x)*x+fstpart+sndpart-np.power(np.e,self.lbd*(-1+self.alpha-self.alpha*x))/self.lbd + np.power(np.e,self.lbd*(-1+self.alpha-self.alpha*breakpoint2))/self.lbd-self.alpha*(np.power(np.e, -self.lbd*(-1+self.alpha+x)/self.alpha) - np.power(np.e, -self.lbd*(-1+self.alpha+breakpoint2)/self.alpha))/self.lbd - x+breakpoint2
      else:
        if x <= breakpoint2:
          return 0
        elif x > breakpoint2 and x <= breakpoint1:
          fstpart = -((1-self.alpha-np.power(np.e,-self.lbd*self.vstar))*breakpoint2)
          return self.interimalloc(x)*x+fstpart-x+breakpoint2-self.alpha*(np.power(np.e, -self.lbd*(-1+self.alpha+x)/self.alpha)-np.power(np.e, -self.lbd*(-1+self.alpha+breakpoint2)/self.alpha)+self.lbd*(breakpoint2-x))/self.lbd
        else:
          fstpart = -((1-self.alpha-np.power(np.e,-self.lbd*self.vstar))*breakpoint2)
          sndpart = -breakpoint1+breakpoint2-self.alpha*(np.power(np.e, -self.lbd*(-1+self.alpha+breakpoint1)/self.alpha)-np.power(np.e, -self.lbd*(-1+self.alpha+breakpoint2)/self.alpha)+self.lbd*(breakpoint2-breakpoint1))/self.lbd
          return self.interimalloc(x)*x+fstpart+sndpart-np.power(np.e,self.lbd*(-1+self.alpha-self.alpha*x))/self.lbd + np.power(np.e,self.lbd*(-1+self.alpha-self.alpha*breakpoint1))/self.lbd-self.alpha*(np.power(np.e, -self.lbd*(-1+self.alpha+x)/self.alpha) - np.power(np.e, -self.lbd*(-1+self.alpha+breakpoint1)/self.alpha))/self.lbd - x+breakpoint1

    def revenue(self):
      x = self.sample(5000000).reshape(-1,2)
      y = x.copy()
      y[:,0] = x[:,1]
      y[:,1] = x[:,0]
      return ((self.phi(x[:,0])-self.alpha*self.phi(x[:,1]))*self.alloc(x)).mean()+((self.phi(x[:,1])-self.alpha*self.phi(x[:,0]))*self.alloc(y)).mean()

    def revenue2(self):
      x = self.sample(10000000)
      ctr = 0
      count = 0
      for i in x:
        ctr += self.interimalloc(i)*self.phi(i)*2
        count += 1
      return ctr/count

In [ ]:
test = ExpoAlloc(1,2,0.75)

In [ ]:
test.revenue2()

In [ ]:
test.revenue()

In [ ]:
test.error(1.2)

In [ ]:
def generate_data(batch_size):
    return np.random.exponential(1,batch_size)

In [ ]:
dt = generate_data(100000).reshape(-1,2)
dt2 = generate_data(100000)
print(test.revenue(dt))
test.revenue2(dt2)

In [ ]:
import pandas as pd
import seaborn as sns
Xpts = np.linspace(0, 2, 101).astype(np.float32)
Ypts = np.linspace(0, 2, 101).astype(np.float32)
X2D,Y2D = np.meshgrid(Ypts,Xpts)
out = np.column_stack((X2D.ravel(),Y2D.ravel()))
Z2D = test.alloc(out)
print(Z2D)
df = pd.DataFrame.from_dict(out)
df['2'] = Z2D
df.columns = ['x','y','z']
df = df.astype(float).round({"x":2, "y":2, "z":3}) 
pivotted= df.pivot('y','x','z')
fig, ax = plt.subplots()
_ = ax.set_xlim([0, 4.0])
_ = ax.set_ylim([0, 4.0])
# print(pivotted)
sns.heatmap(pivotted,cmap='RdBu', rasterized=True)
# plt.axhline(y=1, color='r', linestyle=':')
# ax.plot([0.0,0.8465*50],[0.693*50,0.693*50], color='black')
# ax.plot([0.8465*50,1.5*50],[0.693*50,2*50], color='black')
# ax.plot([0.693*50,0.693*50],[0.0,0.8465*50], color='black')
# ax.plot([0.693*50,2*50],[0.8465*50,1.5*50], color='black')
# ax.text(0.2*50,1.4*50,"Incorrect rec",fontsize='large',color='white')
# ax.text(1.2*50,0.6*50,"Correct rec",fontsize='large', color='white')
# ax.text(0.2*50,1.4*50,"Correct rec",fontsize='large',color='white')
# ax.text(1.2*50,0.6*50,"Incorrect rec",fontsize='large', color='white')
ax.invert_yaxis()
ax.set_xlabel("$v_1$")
ax.set_ylabel("$v_2$")
plt.savefig('analyticala.pdf')
# ax.set_title('Probability of recommending correct action to buyer 1')

In [ ]:
import pandas as pd
import seaborn as sns
fig, ax = plt.subplots()
Xpts = np.linspace(0, 4, 101).astype(np.float32)
y = np.array([])
for x in Xpts:
    y = np.append(y,test.price(x))
    assert abs(test.price(x) - test.scipyprice(x)) < 10e-5
ax.plot(Xpts,y,color="green")
ax.set_xlabel("$v$")
ax.set_ylabel("payment")
# ax.plot([0.693,0.694],[-1,3],":", color='grey')
_ = ax.set_xlim([-0.3, 4.3])
_ = ax.set_ylim([-0.3,2])
def make_square_axes(ax):
    """Make an axes square in screen units.

    Should be called after plotting.
    """
    ax.set_aspect(0.86 / ax.get_data_ratio())
make_square_axes(plt.gca())
# ax.margins(x=0.2)
# fig.set_aspect('equal', adjustable='box')
plt.savefig('analyticalp.pdf')

In [ ]:
from google.colab import files
files.download('analyticala.pdf')
files.download('analyticalp.pdf')

In [ ]:
class Uniform1alloc:
  
    def __init__(self, alpha, thetamax):
      self.alpha = alpha
      self.thetamax = thetamax
      self.vstar = thetamax
    
    def phi(self,x):
      return 2*x-1

    def iphi(self,x):
      return x/2+1/2

    def alloc(self,arr):
      v2r = self.phi(arr[:,1])
      v1r = self.phi(arr[:,0])/self.alpha
      cpr = np.maximum(v1r,self.phi(self.vstar))
      return (v2r<=cpr).astype(int)

    def F(self,x):
      if x < 0.0:
        return 0
      elif x < 1.0:
        return x
      elif x>=1.0:
        return 1
      else:
        print(x,"unexpected CDF value")
    
    def interimalloc(self,x):
       return max(self.F(self.vstar),self.F(self.iphi(self.phi(x)/self.alpha)))+self.alpha*int(x>self.vstar)*self.F(self.iphi(self.alpha*self.phi(x)))-self.alpha
    
    def scipyprice(self,x):
      return x*self.interimalloc(x)-integrate.quad(lambda x: self.interimalloc(x), 0, x)[0]
    
    def error(self,x):
      return integrate.quad(lambda x: self.interimalloc(x), 0, x)[1]
    
    def sample(self, batch_size):
      return np.random.uniform(low = 0.0, high = 1.0, size = (batch_size))

    def price(self,x):
      breakpoint1 = self.vstar
      breakpoint2 = 1/2+1/2*self.alpha*(2*self.vstar-1)
      if breakpoint1 <= breakpoint2:
        if x <= breakpoint1:
          return 0
        elif x > breakpoint1 and x <= breakpoint2:
          fstpart = -((-self.alpha+self.vstar)*breakpoint1)
          return self.interimalloc(x)*x+fstpart+self.alpha*x/2+np.power(self.alpha,2)*x/2-self.vstar*x-np.power(self.alpha,2)*np.power(x,2)/2-self.alpha*breakpoint1/2-np.power(self.alpha,2)*breakpoint1/2+self.vstar*breakpoint1+np.power(self.alpha,2)*np.power(breakpoint1,2)/2
        else:
          fstpart =-((-self.alpha+self.vstar)*breakpoint1)
          sndpart = self.alpha*breakpoint2/2+np.power(self.alpha,2)*breakpoint2/2-self.vstar*breakpoint2-np.power(self.alpha,2)*np.power(breakpoint2,2)/2-self.alpha*breakpoint1/2-np.power(self.alpha,2)*breakpoint1/2+self.vstar*breakpoint1+np.power(self.alpha,2)*np.power(breakpoint1,2)/2
          return self.interimalloc(x)*x+fstpart+sndpart-x/2+x/(2*self.alpha)+self.alpha*x/2+np.power(self.alpha,2)*x/2-np.power(x,2)/(2*self.alpha)-np.power(self.alpha,2)*np.power(x,2)/2+breakpoint2/2-breakpoint2/(2*self.alpha)-(self.alpha*breakpoint2)/2-np.power(self.alpha,2)*breakpoint2/2+np.power(breakpoint2,2)/(2*self.alpha)+np.power(self.alpha,2)*np.power(breakpoint2,2)/2
      else:
        if x <= breakpoint2:
          return 0
        elif x > breakpoint2 and x <= breakpoint1:
          fstpart = -((-self.alpha+self.vstar)*breakpoint2)
          return self.interimalloc(x)*x+fstpart-x/2+x/(2*self.alpha)+self.alpha*x-np.power(x,2)/(2*self.alpha)+breakpoint2/2-breakpoint2/(2*self.alpha)-self.alpha*breakpoint2+np.power(breakpoint2,2)/(2*self.alpha)
        else:
          fstpart = -((-self.alpha+self.vstar)*breakpoint2)
          sndpart = -breakpoint1/2+breakpoint1/(2*self.alpha)+self.alpha*breakpoint1-np.power(breakpoint1,2)/(2*self.alpha)+breakpoint2/2-breakpoint2/(2*self.alpha)-self.alpha*breakpoint2+np.power(breakpoint2,2)/(2*self.alpha)
          return self.interimalloc(x)*x+fstpart+sndpart-x/2+x/(2*self.alpha)+self.alpha*x/2+np.power(self.alpha,2)*x/2-np.power(x,2)/(2*self.alpha)-np.power(self.alpha,2)*np.power(x,2)/2+breakpoint1/2-breakpoint1/(2*self.alpha)-(self.alpha*breakpoint1)/2-np.power(self.alpha,2)*breakpoint1/2+np.power(breakpoint1,2)/(2*self.alpha)+np.power(self.alpha,2)*np.power(breakpoint1,2)/2    
    def revenue(self):
      x = self.sample(5000000).reshape(-1,2)
      y = x.copy()
      y[:,0] = x[:,1]
      y[:,1] = x[:,0]
      return ((self.phi(x[:,0])-self.alpha*self.phi(x[:,1]))*self.alloc(x)).mean()+((self.phi(x[:,1])-self.alpha*self.phi(x[:,0]))*self.alloc(y)).mean()

    def revenue2(self):
      x = self.sample(10000000)
      ctr = 0
      count = 0
      for i in x:
        ctr += self.interimalloc(i)*self.phi(i)*2
        count += 1
      return ctr/count

In [ ]:
test = Uniform1alloc(2.0,0.75)

In [ ]:
test.revenue2()

In [ ]:
test.revenue()

In [ ]:
import pandas as pd
import seaborn as sns
Xpts = np.linspace(0, 1, 401).astype(np.float32)
Ypts = np.linspace(0, 1, 401).astype(np.float32)
X2D,Y2D = np.meshgrid(Ypts,Xpts)
out = np.column_stack((X2D.ravel(),Y2D.ravel()))
Z2D = test.alloc(out)
df = pd.DataFrame.from_dict(out)
df['2'] = Z2D
df.columns = ['x','y','z']
df = df.astype(float).round({"x":3, "y":3, "z":3}) 
pivotted= df.pivot('y','x','z')
fig, ax = plt.subplots()
_ = ax.set_xlim([0, 2.0])
_ = ax.set_ylim([0, 2.0])
# print(pivotted)
sns.heatmap(pivotted,cmap='RdBu', rasterized=True)
# # plt.axhline(y=1, color='r', linestyle=':')
# ax.plot([0.0,0.8465*50],[0.693*50,0.693*50], color='black')
# ax.plot([0.8465*50,1.5*50],[0.693*50,2*50], color='black')
# # ax.plot([0.693*50,0.693*50],[0.0,0.8465*50], color='black')
# # ax.plot([0.693*50,2*50],[0.8465*50,1.5*50], color='black')
# ax.text(0.2*50,1.4*50,"Incorrect rec",fontsize='large',color='white')
# ax.text(1.2*50,0.6*50,"Correct rec",fontsize='large', color='white')
# ax.text(0.2*50,1.4*50,"Correct rec",fontsize='large',color='white')
# ax.text(1.2*50,0.6*50,"Incorrect rec",fontsize='large', color='white')
ax.invert_yaxis()
ax.set_xlabel("$v_1$")
ax.set_ylabel("$v_2$")
plt.savefig('analyticala.pdf')
# ax.set_title('Probability of recommending correct action to buyer 1')

In [ ]:
import pandas as pd
import seaborn as sns
fig, ax = plt.subplots()
Xpts = np.linspace(0, 1, 101).astype(np.float32)
y = np.array([])
for x in Xpts:
    y = np.append(y,test.price(x))
#     assert abs(test.price(x) - test.scipyprice(x)) < 10e-4
ax.plot(Xpts,y,color="green")
ax.set_xlabel("$v$")
ax.set_ylabel("payment")
# ax.plot([0.693,0.694],[-1,3],":", color='grey')
_ = ax.set_xlim([-0.05, 1])
_ = ax.set_ylim([-0.3,1])
def make_square_axes(ax):
    """Make an axes square in screen units.

    Should be called after plotting.
    """
    ax.set_aspect(0.86 / ax.get_data_ratio())
make_square_axes(plt.gca())
# ax.margins(x=0.2)
# fig.set_aspect('equal', adjustable='box')
plt.savefig('analyticalp.pdf')

In [ ]:
class Expo3Alloc:
  
    def __init__(self, alpha, thetamax):
      self.alpha = alpha
      self.thetamax = thetamax
      if thetamax == 0:
        self.vstar = 0
      elif thetamax == 1:
        self.vstar = 1
      if thetamax == 0.5:
        self.vstar = 0.678347
    
    def phi(self,x):
      return x-1

    def F2(self,phix):
      if phix < 0:
        return 0
      else:
        return 1 - np.power(np.e,-phix)-np.power(np.e,-phix)*phix

    def alloc(self,arr):
      v3r = self.phi(arr[:,2])
      v2r = self.phi(arr[:,1])
      v1r = self.phi(arr[:,0])/(self.alpha/2)
      cpr = np.maximum(v1r,self.phi(self.vstar))
      return (v2r+v3r<=cpr).astype(int)
   
    def sample(self,batch_size):
      return np.random.exponential(scale = 1, size = (batch_size))

    def F(self,x):
      if x < 0: return 0
      return 1-np.exp(-x)
    
    def interimalloc(self,x):
       fstterm = max(self.F2(self.vstar),self.F2((2*x-2)/self.alpha+1))
       sndterm = integrate.quad(lambda y: self.F(max(self.phi(self.vstar),self.phi(y)/(self.alpha/2))-self.phi(x)+1)*np.exp(-y), 0, np.inf)[0]
       return fstterm - self.alpha*sndterm

    def interimalloc2(self,x):
       fstterm = max(self.F2(self.vstar),self.F2((2*x-2)/self.alpha+1))
       cutoff = (self.alpha*self.vstar-self.alpha+2)/2
       return fstterm - self.alpha*self.F(cutoff)*self.F(self.phi(self.vstar)-self.phi(x))
    
    def scipyprice(self,x):
      return x*self.interimalloc(x)-integrate.quad(lambda x: self.interimalloc(x), 0, x)[0]
    
    def error(self,x):
      return 0

    def revenue(self):
      x = self.sample(60000000).reshape(-1,3)
      y = x.copy()
      z = x.copy()
      y = y[:,[1,0,2]]
      z = z[:,[2,1,0]]
      return ((self.phi(x[:,0])-self.alpha*self.phi(x[:,1])/2 - self.alpha*self.phi(x[:,2])/2)*self.alloc(x)).mean()+((self.phi(x[:,1])-self.alpha*self.phi(x[:,2])/2 - self.alpha*self.phi(x[:,0])/2)*self.alloc(y)).mean()+((self.phi(x[:,2])-self.alpha*self.phi(x[:,1])/2 - self.alpha*self.phi(x[:,0])/2)*self.alloc(z)).mean()        

In [ ]:
test = Expo3Alloc(0.5,0.5)

In [ ]:
test.revenue()

In [ ]:
import pandas as pd
import seaborn as sns
Xpts = np.linspace(0, 4, 101).astype(np.float32)
Ypts = np.linspace(0, 4, 101).astype(np.float32)
X2D,Y2D = np.meshgrid(Ypts,Xpts)
out = np.column_stack((X2D.ravel(),Y2D.ravel(),Y2D.ravel()))
Z2D = test.alloc(out)
df = pd.DataFrame.from_dict(np.column_stack((X2D.ravel(),Y2D.ravel()*2)))
df['2'] = Z2D
df.columns = ['x','y','z']
df = df.astype(float).round({"x":2, "y":2, "z":3}) 
pivotted= df.pivot('y','x','z')
fig, ax = plt.subplots()
_ = ax.set_xlim([0, 4.0])
_ = ax.set_ylim([0, 4.0])
# print(pivotted)
sns.heatmap(pivotted,cmap='RdBu', rasterized=True)
# plt.axhline(y=1, color='r', linestyle=':')
# ax.plot([0.0,0.8465*50],[0.693*50,0.693*50], color='black')
# ax.plot([0.8465*50,1.5*50],[0.693*50,2*50], color='black')
# ax.plot([0.693*50,0.693*50],[0.0,0.8465*50], color='black')
# ax.plot([0.693*50,2*50],[0.8465*50,1.5*50], color='black')
# ax.text(0.2*50,1.4*50,"Incorrect rec",fontsize='large',color='white')
# ax.text(1.2*50,0.6*50,"Correct rec",fontsize='large', color='white')
# ax.text(0.2*50,1.4*50,"Correct rec",fontsize='large',color='white')
# ax.text(1.2*50,0.6*50,"Incorrect rec",fontsize='large', color='white')
ax.invert_yaxis()
ax.set_xlabel("$v_1$")
ax.set_ylabel("$v_2$")
plt.savefig('analyticala.pdf')
# ax.set_title('Probability of recommending correct action to buyer 1')

In [ ]:
import pandas as pd
import seaborn as sns
fig, ax = plt.subplots()
Xpts = np.linspace(0, 5, 101).astype(np.float32)
y = np.array([])
for x in Xpts:
    y = np.append(y,test.scipyprice(x))
    # assert abs(test.price(x) - test.scipyprice(x)) < 10e-4
ax.plot(Xpts,y,color="green")
ax.set_xlabel("$v$")
ax.set_ylabel("payment")
# ax.plot([0.693,0.694],[-1,3],":", color='grey')
_ = ax.set_xlim([-0.05, 5])
_ = ax.set_ylim([-0.3,4])
def make_square_axes(ax):
    """Make an axes square in screen units.

    Should be called after plotting.
    """
    ax.set_aspect(0.86 / ax.get_data_ratio())
make_square_axes(plt.gca())
# ax.margins(x=0.2)
# fig.set_aspect('equal', adjustable='box')
plt.savefig('analyticalp.pdf')

In [ ]:
import collections.abc

In [ ]:
class AsymUniform:
  
    def __init__(self, alpha, thetamax):
      self.alpha = alpha
      self.thetamax = thetamax
      self.vstar1 = thetamax
      self.vstar2 = 2*thetamax
    
    def phi1(self,x):
      return 2*x-1

    def phi2(self,x):
      return 2*x-2

    def iphi1(self,x):
      return x/2+1/2
    
    def iphi2(self,x):
      return x/2+1

    def alloc1(self,arr):
      v2r = self.phi2(arr[:,1])
      v1r = self.phi1(arr[:,0])/self.alpha
      cpr = np.maximum(v1r,self.phi2(self.vstar2))
      return (v2r<=cpr).astype(int)
    
    def alloc2(self,arr):
      v1r = self.phi1(arr[:,0])
      v2r = self.phi2(arr[:,1])/self.alpha
      cpr = np.maximum(v2r,self.phi1(self.vstar1))
      return (v1r<=cpr).astype(int)

    def F1(self,x):
      if x < 0:
        return 0
      elif x < 1:
        return x
      else:
        return 1
    
    def F2(self,x):
      if x < 0:
        return 0
      elif x < 2:
        return x/2
      else:
        return 1
    
    def interimalloc1(self,x):
       return max(self.F2(self.vstar1),self.F2(self.iphi1(self.phi1(x)/self.alpha)))+self.alpha*int(x>self.vstar2)*self.F1(self.iphi1(self.alpha*self.phi1(x)))-self.alpha

    def interimalloc2(self,x):
       return max(self.F1(self.vstar2),self.F1(self.iphi2(self.phi2(x)/self.alpha)))+self.alpha*int(x>self.vstar1)*self.F2(self.iphi2(self.alpha*self.phi2(x)))-self.alpha
    
    def scipyprice1(self,x):
      return x*self.interimalloc1(x)-integrate.quad(lambda x: self.interimalloc(x), 0, x)[0]
    
    def scipyprice2(self,x):
      return x*self.interimalloc2(x)-integrate.quad(lambda x: self.interimalloc(x), 0, x)[0]
    
    def error(self,x):
      return integrate.quad(lambda x: self.interimalloc1(x), 0, x)[1]
    
    def sample(self,idx,batchsize):
      return np.random.uniform(low = 0.0, high = idx + 1.0, size = (batchsize))

    def revenue(self):
      x = self.sample(0,50000000)
      y = self.sample(1,50000000)
      x = np.stack((x,y),-1)
      return ((self.phi1(x[:,0])-self.alpha*self.phi2(x[:,1]))*self.alloc1(x)).mean()+((self.phi2(x[:,1])-self.alpha*self.phi1(x[:,0]))*self.alloc2(x)).mean()

    def revenue2(self):
      x = self.sample(0,1000000)
      y = self.sample(1,1000000)
      ctr = 0
      count = 0
      for ind in range(len(x)):
        i = x[ind]
        j = y[ind]
        ctr += self.interimalloc1(i)*self.phi1(i) + self.interimalloc2(j)*self.phi2(j)
        count += 1
      return ctr/count

In [ ]:
test = AsymUniform(0.5,0.75)

In [ ]:
test.revenue()

In [ ]:
test.revenue2()

In [ ]:
import pandas as pd
import seaborn as sns
Xpts = np.linspace(0, 2, 401).astype(np.float32)
Ypts = np.linspace(0, 1, 401).astype(np.float32)
X2D,Y2D = np.meshgrid(Ypts,Xpts)
out = np.column_stack((X2D.ravel(),Y2D.ravel()))
Z2D = test.alloc1(out)
df = pd.DataFrame.from_dict(out)
df['2'] = Z2D
df.columns = ['x','y','z']
df = df.astype(float).round({"x":3, "y":3, "z":3}) 
pivotted= df.pivot('y','x','z')
fig, ax = plt.subplots()
_ = ax.set_xlim([0, 2.0])
_ = ax.set_ylim([0, 2.0])
# print(pivotted)
sns.heatmap(pivotted,cmap='RdBu', rasterized=True)
# # plt.axhline(y=1, color='r', linestyle=':')
# ax.plot([0.0,0.8465*50],[0.693*50,0.693*50], color='black')
# ax.plot([0.8465*50,1.5*50],[0.693*50,2*50], color='black')
# # ax.plot([0.693*50,0.693*50],[0.0,0.8465*50], color='black')
# # ax.plot([0.693*50,2*50],[0.8465*50,1.5*50], color='black')
# ax.text(0.2*50,1.4*50,"Incorrect rec",fontsize='large',color='white')
# ax.text(1.2*50,0.6*50,"Correct rec",fontsize='large', color='white')
# ax.text(0.2*50,1.4*50,"Correct rec",fontsize='large',color='white')
# ax.text(1.2*50,0.6*50,"Incorrect rec",fontsize='large', color='white')
ax.invert_yaxis()
ax.set_xlabel("$v_1$")
ax.set_ylabel("$v_2$")
plt.savefig('analyticala.pdf')
# ax.set_title('Probability of recommending correct action to buyer 1')

In [ ]:
import pandas as pd
import seaborn as sns
Xpts = np.linspace(0, 2, 401).astype(np.float32)
Ypts = np.linspace(0, 1, 401).astype(np.float32)
X2D,Y2D = np.meshgrid(Ypts,Xpts)
out = np.column_stack((X2D.ravel(),Y2D.ravel()))
Z2D = test.alloc2(out)
df = pd.DataFrame.from_dict(out)
df['2'] = Z2D
df.columns = ['x','y','z']
df = df.astype(float).round({"x":3, "y":3, "z":3}) 
pivotted= df.pivot('y','x','z')
fig, ax = plt.subplots()
_ = ax.set_xlim([0, 2.0])
_ = ax.set_ylim([0, 2.0])
# print(pivotted)
sns.heatmap(pivotted,cmap='RdBu', rasterized=True)
# # plt.axhline(y=1, color='r', linestyle=':')
# ax.plot([0.0,0.8465*50],[0.693*50,0.693*50], color='black')
# ax.plot([0.8465*50,1.5*50],[0.693*50,2*50], color='black')
# # ax.plot([0.693*50,0.693*50],[0.0,0.8465*50], color='black')
# # ax.plot([0.693*50,2*50],[0.8465*50,1.5*50], color='black')
# ax.text(0.2*50,1.4*50,"Incorrect rec",fontsize='large',color='white')
# ax.text(1.2*50,0.6*50,"Correct rec",fontsize='large', color='white')
# ax.text(0.2*50,1.4*50,"Correct rec",fontsize='large',color='white')
# ax.text(1.2*50,0.6*50,"Incorrect rec",fontsize='large', color='white')
ax.invert_yaxis()
ax.set_xlabel("$v_1$")
ax.set_ylabel("$v_2$")
plt.savefig('analyticala.pdf')
# ax.set_title('Probability of recommending correct action to buyer 1')

In [ ]:
import pandas as pd

In [ ]:
class Irregular:
  
    def __init__(self, alpha, thetamax):
      self.alpha = alpha
      self.thetamax = thetamax
    
    def phi(self,x):
      mask1 = ( x<= (7.0 - np.sqrt(5.0))/2.0)
      mask2 = np.logical_and((x>(7.0 - np.sqrt(5.0))/2.0), (x<=(11.0 - np.sqrt(5.0))/2.0))
      mask3 = np.logical_and((x>(7.0 - np.sqrt(5.0))/2.0), (x>(11.0 - np.sqrt(5.0))/2.0))
      output = x.copy()
      output[mask1] = 2.0 * x[mask1] - 4.0
      output[mask2] = 3.0 - np.sqrt(5.0)
      output[mask3] = 2.0 * x[mask3] - 8.0
      return output

    def scalarphi(self,x):
      if(x <= (7.0 - np.sqrt(5.0))/2.0):
        return 2.0 * x - 4.0
      elif(x <= (11.0 - np.sqrt(5.0))/2.0):
        return 3.0 - np.sqrt(5.0)
      else:
        return 2.0 * x - 8.0

    def Fphi(self,x):
      inv = self.iphi(x)
      return self.F(inv)

    def F(self,x):
      if x < 0:
        return 0
      elif x <3:
        return 0.75 * (x)/3
      elif x <= 8:
        return 0.75 + ((x-3)/5)*0.25
      else:
        # print(x,"invalid CDF value")
        return 1
    
    def iF(self,x):
      if x <= 0.75:
        return 3*x/0.75
      else:
        return (x-0.75)*20+3

    def iphi(self,x):
      if x < 3.0 - np.sqrt(5.0):
        return (x+4)/2
      elif x > 3.0 - np.sqrt(5.0):
        return (x+8)/2
      else:
        # print("inverse not properly defined")
        return (9.0 - np.sqrt(5.0))/2.0
    
    def iphiv(self,x):
      mask1 = ( x <= 3.0 - np.sqrt(5.0))
      mask2 = (x > 3.0 - np.sqrt(5.0))
      output = x.copy()
      output[mask1] = (output[mask1]+4)/2
      output[mask2] = (output[mask2]+8)/2
      return output
    
    def alloc(self,arr):
      vstar = self.iF(self.thetamax)
      v2r = arr[:,1]
      v1r = self.phi(arr[:,0])/self.alpha
      cpr = np.maximum(self.iphiv(v1r),vstar)

#       ## convert your array into a dataframe
#       df = pd.DataFrame (arr)

#       ## save to xlsx file

#       filepath = 'file1.xlsx'

#       df.to_excel(filepath, index=False)
#       df = pd.DataFrame ((v2r<=cpr).astype(int))

#       ## save to xlsx file

#       filepath = 'file2.xlsx'

#       df.to_excel(filepath, index=False)
      return (v2r<=cpr).astype(int)

#     def alloc2(self,arr):
#       vstar = self.iF(self.thetamax)
#       v2r = self.phi(arr[:,1])
#       v1r = self.phi(arr[:,0])/self.alpha
#       cpr = np.maximum(v1r,self.scalarphi(vstar))
#       return (v2r<=cpr).astype(int)
    
    def interimalloc(self,x):
      vstar = self.iF(self.thetamax)
      fstterm = max(self.F(vstar),self.F(self.iphi(self.scalarphi(x)/self.alpha)))
      return fstterm+self.alpha*int(x>vstar)*self.F(self.iphi(self.alpha*self.scalarphi(x)))-self.alpha

    def interimalloc2(self,x):
      vstar = self.iF(self.thetamax)
      fstterm = max(self.Fphi(self.scalarphi(vstar)),self.Fphi(self.scalarphi(x)/self.alpha))
      return fstterm+self.alpha*int(x>vstar)*self.Fphi(self.alpha*self.scalarphi(x))-self.alpha
    
    def scipyprice(self,x):
      return x*self.interimalloc(x)-integrate.quad(lambda x: self.interimalloc(x), 0, x,limit=150)[0]
        
    def error(self,x):
      return integrate.quad(lambda x: self.interimalloc(x), 0, x,limit=150)[1]

    def revenue(self,x):
      y = x.copy()
      y[:,0] = x[:,1]
      y[:,1] = x[:,0]
      return ((self.phi(x[:,0])-self.alpha*self.phi(x[:,1]))*self.alloc(x)).mean()+((self.phi(x[:,1])-self.alpha*self.phi(x[:,0]))*self.alloc(y)).mean()

    def revenue2(self,x):
      ctr = 0
      count = 0
      for i in x:
        ctr += self.interimalloc(i)*self.scalarphi(i)*2
        count += 1
      return ctr/count

In [ ]:
test = Irregular(2.0,0.5)

In [ ]:
import pandas as pd
import seaborn as sns
Xpts = np.linspace(0, 8, 401).astype(np.float32)
Ypts = np.linspace(0, 8, 401).astype(np.float32)
X2D,Y2D = np.meshgrid(Ypts,Xpts)
out = np.column_stack((X2D.ravel(),Y2D.ravel()))
Z2D = test.alloc(out)
df = pd.DataFrame.from_dict(out)
df['2'] = Z2D
df.columns = ['x','y','z']
df = df.astype(float).round({"x":3, "y":3, "z":3}) 
pivotted= df.pivot('y','x','z')
fig, ax = plt.subplots()
_ = ax.set_xlim([0, 8.0])
_ = ax.set_ylim([0, 8.0])
# print(pivotted)
sns.heatmap(pivotted,cmap='RdBu', rasterized=True)
# # plt.axhline(y=1, color='r', linestyle=':')
# ax.plot([0.0,0.8465*50],[0.693*50,0.693*50], color='black')
# ax.plot([0.8465*50,1.5*50],[0.693*50,2*50], color='black')
# # ax.plot([0.693*50,0.693*50],[0.0,0.8465*50], color='black')
# # ax.plot([0.693*50,2*50],[0.8465*50,1.5*50], color='black')
# ax.text(0.2*50,1.4*50,"Incorrect rec",fontsize='large',color='white')
# ax.text(1.2*50,0.6*50,"Correct rec",fontsize='large', color='white')
# ax.text(0.2*50,1.4*50,"Correct rec",fontsize='large',color='white')
# ax.text(1.2*50,0.6*50,"Incorrect rec",fontsize='large', color='white')
ax.invert_yaxis()
ax.set_xlabel("$v_1$")
ax.set_ylabel("$v_2$")
plt.savefig('analyticala.pdf')
# ax.set_title('Probability of recommending correct action to buyer 1')

In [ ]:
import pandas as pd
import seaborn as sns
fig, ax = plt.subplots()
Xpts = np.linspace(0, 8, 201).astype(np.float32)
y = np.array([])
for x in Xpts:
    y = np.append(y,test.interimalloc(x))
    # assert abs(test.price(x) - test.scipyprice(x)) < 10e-4
ax.plot(Xpts,y,color="green")
ax.set_xlabel("$v$")
ax.set_ylabel("payment")
# ax.plot([0.693,0.694],[-1,3],":", color='grey')
_ = ax.set_xlim([-0.05, 8])
_ = ax.set_ylim([-0.3,1])
def make_square_axes(ax):
    """Make an axes square in screen units.

    Should be called after plotting.
    """
    ax.set_aspect(0.86 / ax.get_data_ratio())
make_square_axes(plt.gca())
# ax.margins(x=0.2)
# fig.set_aspect('equal', adjustable='box')
plt.savefig('analyticalp.pdf')

In [ ]:
def samplers(batch_size):
    spl1 = np.random.uniform(low = 0.0, high = 3.0, size = (batch_size))
    spl2 = np.random.uniform(low = 3.0, high = 8.0, size = (batch_size))
    choice = np.random.rand(batch_size)
    choice1 = (choice>0.75)
    result = spl1.copy()
    result[choice1] = spl2[choice1]
    return result

In [ ]:
spl = samplers(1000000)
ctr = 0
err = 0
for item in spl:
  ctr += 2*test.scipyprice(item)
  err += test.error(item)
print(ctr/10000,err/10000)

In [ ]:
spl = samplers(3000000)
test.revenue2(spl)

In [ ]:
spl = samplers(1000000).reshape(-1,2)
test.revenue(spl)

In [ ]:
spl = samplers(10000).reshape(-1,2)
ctr = 0
for item in spl:
  ctr += 2*test.scipyprice(item)